# Explore here

## 1) Import required libraries

In [46]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## 2) Load dataset

In [47]:
recipes_df = pd.read_csv('../data/csv/epi_r.csv')

## 3) Visualize dataset information

In [48]:
print(f'🔹 Dataset shape: {recipes_df.shape}')
print(f'🔹 Dataset information: ')
recipes_df.describe()

🔹 Dataset shape: (20052, 680)
🔹 Dataset information: 


,rating,calories,protein,fat,sodium,#cakeweek,#wasteless,22-minute meals,3-ingredient recipes,30 days of groceries,...,yellow squash,yogurt,yonkers,yuca,zucchini,cookbooks,leftovers,snack,snack week,turkey
count,20052.000000,1.593500e+04,15890.000000,1.586900e+04,1.593300e+04,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,...,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000
mean,3.714467,6.322958e+03,100.160793,3.468775e+02,6.225975e+03,0.000299,0.000050,0.000848,0.001346,0.000349,...,0.001247,0.026332,0.000050,0.000299,0.014861,0.000150,0.000349,0.001396,0.000948,0.022741
std,1.340829,3.590460e+05,3840.318527,2.045611e+04,3.333182e+05,0.017296,0.007062,0.029105,0.036671,0.018681,...,0.035288,0.160123,0.007062,0.017296,0.121001,0.012231,0.018681,0.037343,0.030768,0.149080
min,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.750000,1.980000e+02,3.000000,7.000000e+00,8.000000e+01,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4.375000,3.310000e+02,8.000000,1.700000e+01,2.940000e+02,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,4.375000,5.860000e+02,27.000000,3.300000e+01,7.110000e+02,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,5.000000,3.011122e+07,236489.000000,1.722763e+06,2.767511e+07,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### Feature required before proceeding
As dataset contains several dimensions (680 colums), selecting the most relevant features is essential for visualization and preprocessing:

The projects further proceeds dividing dataset into 2 separate dataframes before rejoining:
- **Dataframe with nutritional values and diet labels**
- **Dataframe with most 150 frequent ingredients**

#### Creating Nutrition Dataframe

In [49]:
nutrition_df = recipes_df[['title', 'calories', 'protein', 'fat', 'sodium', 'vegan', 'vegetarian', 'wheat/gluten-free', 'no sugar added']]
nutrition_df.head()

##

,title,calories,protein,fat,sodium,vegan,vegetarian,wheat/gluten-free,no sugar added
0,"Lentil, Apple, and Turkey Wrap",426.0,30.0,7.0,559.0,0.0,0.0,0.0,0.0
1,Boudin Blanc Terrine with Red Onion Confit,403.0,18.0,23.0,1439.0,0.0,0.0,0.0,0.0
2,Potato and Fennel Soup Hodge,165.0,6.0,7.0,165.0,0.0,0.0,0.0,0.0
3,Mahi-Mahi in Tomato Olive Sauce,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
4,Spinach Noodle Casserole,547.0,20.0,32.0,452.0,0.0,1.0,0.0,0.0


To complete macronutrient profile of each recipe, the **carbohydrates feature** (not present in dataframe) shall be derived (knowing that C 4 kcal/g, P 4 kcal/g, F 9 kcal/g and Kcal=P+C+F)

In [50]:
nutrition_df = nutrition_df.dropna()
nutrition_df['carbohydrates'] = ((nutrition_df['calories'] - (nutrition_df['protein'] * 4) - (nutrition_df['fat'] * 9)) / 4)
# Aply .clip(lower=0) to avoid negative values
nutrition_df['carbohydrates'] = nutrition_df['carbohydrates'].clip(lower=0).astype('float')
# Reorder columns
nutrition_df = nutrition_df[[ 'title', 'calories', 'protein', 'carbohydrates', 'fat', 'sodium', 'vegetarian', 'wheat/gluten-free', 'no sugar added']]
nutrition_df.head()

,title,calories,protein,carbohydrates,fat,sodium,vegetarian,wheat/gluten-free,no sugar added
0,"Lentil, Apple, and Turkey Wrap",426.0,30.0,60.75,7.0,559.0,0.0,0.0,0.0
1,Boudin Blanc Terrine with Red Onion Confit,403.0,18.0,31.00,23.0,1439.0,0.0,0.0,0.0
2,Potato and Fennel Soup Hodge,165.0,6.0,19.50,7.0,165.0,0.0,0.0,0.0
4,Spinach Noodle Casserole,547.0,20.0,44.75,32.0,452.0,1.0,0.0,0.0
5,The Best Blts,948.0,19.0,40.25,79.0,1042.0,0.0,0.0,0.0


To derive nutrition tags (High Protein, Low Carb, Is Balanced), macronutrient dimensions are used (calories, protein, carbohydrates, fat)
- **High Protein**: 40% or more of calories coming from protein.
- **Low Carb**: 10% or less of calories coming from carbohydrates.
- **Is Balanced**: about 40% calories coming from protein, about 30% calories coming from carbohydrates, about 30% calories coming from fat .